In [14]:
!pip install --quiet -U langchain langchain-openai
%pip install --quiet --upgrade langchain langchain-community langchain-chroma


Note: you may need to restart the kernel to use updated packages.


In [15]:
!pip install --quiet supabase

In [16]:
import os
from supabase import create_client, Client

openai_api_key = os.getenv("OPENAI_API_KEY_1010")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

print(f"OpenAI API: {openai_api_key[:4]}********")
print(f"LangChain API Key: {langchain_api_key[:4]}********")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["LANGCHAIN_PROJECT"] = "tech4impact_1010"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"

SUPABASE_URL = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
SUPABASE_KEY = os.getenv("NEXT_PUBLIC_SUPABASE_SERVICE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)



OpenAI API: sk-p********
LangChain API Key: lsv2********


In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

2024-11-08 16:17:57,272 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-96d7415e-bc7a-40e8-9426-f6af643436e6-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

In [18]:
import os
import json
import getpass
import logging 
# Set your OpenAI API key
#os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA



In [19]:
# 함수 정의: 시간 문자열을 밀리초로 변환
def time_str_to_ms(time_str):
    match = re.match(r'(\d+)분 (\d+)초', time_str)
    if match:
        minutes = int(match.group(1))
        seconds = int(match.group(2))
        return (minutes * 60 + seconds) * 1000
    else:
        logging.warning(f"시간 형식이 일치하지 않습니다: {time_str}")
        return 0

# 함수 정의: 밀리초를 시간 문자열로 변환
def ms_to_minutes_seconds_str(milliseconds):
    minutes = milliseconds // 60000
    seconds = (milliseconds % 60000) // 1000
    return f"{minutes}분 {seconds}초"


with open('transcription.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

utterances = data['utterances']

n = 3941239

# 발화를 Document 객체로 생성
filtered_documents = [
    Document(
        page_content=utterance['msg'],
        metadata={
            'start_at': utterance['start_at'],
            'duration': utterance['duration'],
            'speaker': utterance['spk'],
            'speaker_type': utterance['spk_type']
        }
    )
    for utterance in utterances if utterance['start_at'] > n
]
logging.info(f"총 {len(filtered_documents)}개의 문서 생성.")

# 시작 시간 기준으로 정렬
documents = sorted(filtered_documents, key=lambda doc: doc.metadata['start_at'])


2024-11-08 16:17:57,316 - INFO - 총 403개의 문서 생성.


In [20]:
full_text = ''
for doc in documents:
    start_time_str = ms_to_minutes_seconds_str(doc.metadata['start_at'])
    full_text += f"[{start_time_str}] {doc.page_content}\n"

# 언어 모델 초기화 # mini 버전 써보기
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)



# 임베딩 및 벡터스토어 초기화
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings, persist_directory=None)

# 리트리버 
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


2024-11-08 16:17:59,519 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [25]:
def extract_topics(full_text, llm):
    prompt = """
당신은 상담 세션의 스크립트에서 주요 토픽을 추출하는 AI입니다. 스크립트 내용을 기반으로 주요 토픽을 시간 순서에 따라 나열해 주세요. 각 토픽은 다음 형식으로 작성해 주세요:

**토픽 1:**
시작 시간: X분 Y초
종료 시간: A분 B초
내용: 주요 토픽 내용

**토픽 2:**
시작 시간: ...
종료 시간: ...
내용: ...

스크립트:
{text}

주요 토픽 목록:
"""
    messages = [
        {"role": "system", "content": "당신은 상담 스크립트에서 주요 토픽을 추출하는 AI입니다."},
        {"role": "user", "content": prompt.format(text=full_text)}
    ]

    try:

        response = llm.invoke(messages)

        topics_text = response['choices'][0]['message']['content'].strip()
        
        # 간단한 파싱 로직
        topics = []
        topic_pattern = r"토픽\s*\d+:\s*시작 시간:\s*(\d+분 \d+초)\s*종료 시간:\s*(\d+분 \d+초)\s*내용:\s*(.*)"
        matches = re.findall(topic_pattern, topics_text, re.DOTALL)
        for match in matches:
            start_time, end_time, content = match
            topics.append((start_time.strip(), end_time.strip(), content.strip()))
        logging.info(f"추출된 토픽 수: {len(topics)}")
        return topics
    except Exception as e:
        logging.error(f"Error extracting topics: {e}")
        return []


In [26]:
llm = ChatOpenAI(
                        model="gpt-4o",
                         temperature=0,
            max_tokens=1000,
        )

extract_topics(full_text,llm)

2024-11-08 16:22:16,716 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-08 16:22:16,720 - ERROR - Error extracting topics: 'AIMessage' object is not subscriptable


[]